# Math 542A Homework 6

In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as sp

### Problem 2

In [2]:
iris = pd.read_excel("C:\\Users\\mbadi\\iris_data.xlsx") #50 observations per class

In [3]:
set(iris["class"])

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [4]:
iris.head() #Displays first five observation vectors. 

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


We'll proceed to hard-code the solutions instead of using functions. 

#### (a) Test the hypothesis that the mean vector of petal and sepal length is equal for versicolor and virginica.

In [5]:
versi_lens = iris[iris["class"]=="Iris-versicolor"].iloc[:,[0,2]]

In [6]:
virgi_lens = iris[iris["class"]=="Iris-virginica"].iloc[:,[0,2]]

In [7]:
x_bar_versi = np.mean([versi_lens.iloc[:,0],versi_lens.iloc[:,1]],axis=1)
x_bar_virgi = np.mean([virgi_lens.iloc[:,0],virgi_lens.iloc[:,1]],axis=1)

In [8]:
S_versi = np.cov([versi_lens.iloc[:,0],versi_lens.iloc[:,1]],rowvar=True)
S_virgi = np.cov([virgi_lens.iloc[:,0],virgi_lens.iloc[:,1]],rowvar=True)

In [14]:
W = np.matrix((49)*S_versi + (49)*S_virgi)

We pooled the covariance matrices, since they're compatible. 

In [12]:
x_bar = 50*(x_bar_versi + x_bar_virgi)/100

In [13]:
B = 50*np.matmul(np.transpose(np.matrix(x_bar_versi-x_bar)),np.matrix(x_bar_versi-x_bar)) + 50*np.matmul(np.transpose(np.matrix(x_bar_virgi-x_bar)),np.matrix(x_bar_virgi-x_bar))

In [15]:
Wilks_lambda = np.linalg.det(W)/np.linalg.det(B+W)

In [16]:
test_statistic = 97*(1-math.sqrt(Wilks_lambda))/math.sqrt(Wilks_lambda)

In [17]:
f_value = sp.f.ppf(0.95,2,97) #alpha = 0.05

In [18]:
(test_statistic,f_value)

(79.30164464023727, 3.090186675154859)

##### Since the test statistic for the case of two variables and two groups exceeds the respective 95% confidence f-tail, we have sufficient evidence to reject the null hypothesis of equality of means. Hence, the mean of either sepal length or petal length differs significantly between versicolor and virginica. 

#### b)  Test the hypothesis that the mean vector of petal and sepal length is equal across all species.

In [77]:
setos_lens = iris[iris["class"]=="Iris-setosa"].iloc[:,[0,2]]

In [78]:
x_bar_setos = np.mean([setos_lens.iloc[:,0],setos_lens.iloc[:,1]],axis=1)

In [79]:
S_setos = np.cov([setos_lens.iloc[:,0],setos_lens.iloc[:,1]],rowvar=True)

In [84]:
W_all = np.matrix(49*S_versi + 49*S_virgi + 49*S_setos)

In [83]:
x_bar_all = 50*(x_bar_versi + x_bar_virgi + x_bar_setos)/150

In [85]:
B_all = 50*np.matmul(np.transpose(np.matrix(x_bar_versi-x_bar_all)),np.matrix(x_bar_versi-x_bar_all)) + 50*np.matmul(np.transpose(np.matrix(x_bar_virgi-x_bar_all)),np.matrix(x_bar_virgi-x_bar_all)) + 50*np.matmul(np.transpose(np.matrix(x_bar_setos-x_bar_all)),np.matrix(x_bar_setos-x_bar_all))

In [31]:
W_lambda = np.linalg.det(W)/np.linalg.det(B+W)  #Wilks lambda

In [89]:
test_stat = 73*(1-math.sqrt(W_lambda))/math.sqrt(W_lambda)

In [90]:
f_val = sp.f.ppf(.95,4,146)

In [91]:
(test_stat,f_val)

(59.68061916224042, 2.433633447712136)

##### Same result as in the case with only two groups; we have sufficient evidence to reject the null hypothesis, so that the vector means of sepal and petal length are significantly different between at least two of the three groups. 

#### c) Build confidence intervals for the mean vector of petal and sepal length for visicolor.

##### We'll proceed to construct T-squared confidence intervals for the means of petal and sepal length for versicolor. We'll recycle the T-squared function from the previous homework.

In [25]:
def T_squared_interval(x,alpha): #for x univariate
    x = np.array(x)
    x_bar = np.mean(x,axis=0)
    n = len(x)
    s = np.std(x)
    
    coeff = (2*(n-1))/(n*(n-2))
    f_val = sp.f.ppf(1-alpha,2,n-2)
    
    lower_bound = x_bar - math.sqrt(coeff*f_val)*s/math.sqrt(n)
    upper_bound = x_bar + math.sqrt(coeff*f_val)*s/math.sqrt(n)
    
    return(lower_bound,upper_bound)

In [26]:
T_squared_interval(versi_lens.iloc[:,0],0.05)   #alpha = 0.05

(5.909916010068213, 5.962083989931787)

##### ^Confidence interval for versicolor sepal length.

In [27]:
T_squared_interval(versi_lens.iloc[:,1],0.05)

(4.236253703313551, 4.283746296686449)

##### ^Confidence interval for versicolor petal length.

#### d) Build confidence intervals for the difference mean vector of petal and sepal length for visicolor and virginica.

In [46]:
def Mean_difference_confidence_interval(x1,x2,i,alpha):  #i denotes s_ii
    x1 = np.array(x1)
    x2 = np.array(x2)
    
    n1 = len(x1)
    n2 = len(x2)
    
    x1_bar = np.mean(x1)
    x2_bar = np.mean(x2)
    
    S_pooled = ((n1-1)/(n1+n2-1))*np.cov(x1,rowvar=False) + ((n2-1)/(n1+n2-1))*np.cov(x2,rowvar=False)
    
    c_squared = ((n1+n2-2)*2/(n1+n2-3))*sp.f.ppf(1-alpha,2,n1+n2-3)
    
    upper_bound = (x1_bar-x2_bar) + math.sqrt(c_squared)*math.sqrt(((1/n1)+(1/n2))*S_pooled[i,i])
    lower_bound = (x1_bar-x2_bar) - math.sqrt(c_squared)*math.sqrt(((1/n1)+(1/n2))*S_pooled[i,i])
    
    return(lower_bound,upper_bound)

In [43]:
a=np.array([[versi_lens.iloc[i,0],versi_lens.iloc[i,1]] for i in range(len(versi_lens))])

In [48]:
b = np.array([[virgi_lens.iloc[i,0],virgi_lens.iloc[i,1]] for i in range(len(versi_lens))])

In [50]:
Mean_difference_confidence_interval(a,b,0,.05)

(-1.2599609275655346, -0.6840390724344663)

##### ^Confidence interval for the difference of sepal length means between versicolor and virginica.

In [51]:
Mean_difference_confidence_interval(a,b,1,.05)

(-1.2268541358466598, -0.717145864153341)

##### ^Confidence interval for the difference of petal length means between versicolor and virginica. 